In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming= True
)




In [10]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


In [11]:
# system : 주로 배열로 대답하세요, 당신은 어떤 역할을 부여받았습니다 등으로 대답의 틀을 만들어준다.
# human : 실제 질문

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

In [12]:
# outputParser : AI로부터 받은 응답을 변형

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question" : "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [13]:
chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are chef"),
        ("human", "I want {cuisine} food recipe")
    ]
)

veg_chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "youa are vegetarian chef"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_template | chat
veg_chain = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

AIMessage(content="I see that you provided a recipe for Chicken Tikka Masala, which is a popular Indian dish. However, as a vegetarian chef, I don't cook with chicken. Would you like me to provide you with a vegetarian version of this dish or suggest a different vegetarian recipe instead? Let me know how I can assist you further!")

In [14]:
program_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 쓰는 시인이야. 특히 프로그래밍 언어에 대해 잘 알고 있고 그것을 가지고 시를 아주 잘 써"),
        ("human", "{program_lang}으로 프로그래밍 언어에 관한 시를 써줘")
    ]
)

talk_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 받으면 그 시에 대해 화자의 심정, 비유법 등에 대해 설명하는 선생님이야."),
        ("human", "{poem}에 대해 설명해줘")
    ]
)

program_chain = program_template | chat
talk_chain = talk_template | chat

speaker_chain = {"poem" : program_chain} | talk_chain

speaker_chain.invoke({
    "program_lang" : "java"
})

AIMessage(content='이 시는 자바 프로그래밍에 대한 열정과 매력을 담고 있는 것 같네요. 시인은 한 줄 한 줄 코딩을 펼치며 자바의 세계에 빠져든다고 표현하면서, 클래스와 객체가 춤을 추며 무한한 가능성을 보여준다고 표현하고 있어요. 이는 자바 프로그래밍에서 객체지향적인 접근법과 뛰어난 확장성을 감탄하는 것으로 해석될 수 있겠네요.\n\n또한, 컴파일러의 소리와 실행 결과가 화면에 비춰지는 장면을 통해 프로그래밍의 과정을 생생하게 묘사하고 있습니다. 에러와 예외를 만나도 결국은 해결책을 찾아낸다는 부분은 프로그래밍 과정에서 마주치는 어려움을 극복하며 성장하는 과정을 담은 것 같아요.\n\n마지막으로, 시인은 자바의 매력에 빠져 계속해서 공부하고 싶다고 표현하며, 자신의 작은 작품을 만들어내는 즐거움을 느낀다고 말하고 있습니다. 이는 자바 프로그래밍을 통해 창의적인 작품을 만들어내는 즐거움과 성취감을 경험하는 것을 의미할 수 있겠네요. 전체적으로 이 시는 자바 프로그래밍에 대한 열정과 즐거움을 담고 있는 멋진 작품이라고 할 수 있겠네요.')

In [9]:
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,    
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Germany"})

AIMessageChunk(content='AI:\n            I know this:\n            Capital: Berlin\n            Language: German\n            Food: Bratwurst and Sauerkraut\n            Currency: Euro')

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [ ]:
# 예제 랜덤 생성기 

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

In [ ]:
# 영화 정보 답장 프롬프트

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

movie_examples = [
    {
        "movie" : "극한직업",
        "answer": """
            감독은 이병헌입니다.
            주요 출연진은 류승룡, 이하늬 등이 출연합니다.
            개봉일은 2019년 1월 23일입니다.
            장르는 코미디와 액션입니다. 
        """
    },
    {
        "movie" : "범죄도시",
        "answer": """
            감독은 강윤성입니다.
            주요 출연진은 마동석, 윤계상 등이 출연합니다.
            개봉일은 2017년 10월 3일입니다.
            장르는 범죄와 액션입니다. 
        """
    },
    {
        "movie" : "인사이드아웃2",
        "answer": """
            감독은 Kelsey Mann입니다.
            주요 출연진은 라일리, 감정들 등이 출연합니다.
            개봉일은 2024년 6월 14일입니다.
            장르는 애니메이션입니다. 
        """
    },
]

movie_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}에 대한 영화 정보를 알려줘"),
        ("ai", "{answer}")
    ]
)

movie_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=movie_example_prompt,
    examples=emoji_movie_examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 정보에 대해 잘 알고 있어"),
        movie_example_prompt,
        ("human", "{movie}에 대한 영화 정보를 알려줘")
    ]
)

movie_chain = final_prompt | chat

movie_chain.invoke({"movie": "아이언맨"})

"Iron Man"은 2008년에 개봉한 마블 시네마틱 유니버스의 첫 번째 영화로, 로버트 다우니 주니어가 토니 스타크/아이언맨 역으로 출연했습니다. 이 영화는 토니 스타크가 철갑수트를 만들어 슈퍼히어로로 변신하는 이야기를 다루고 있습니다. 감독은 존 파브로가 맡았으며, 영화는 큰 성공을 거두었고 마블 시네마틱 유니버스의 시작을 알린 작품으로 손꼽힙니다. 아이언맨은 많은 팬들에게 사랑받는 마블의 대표적인 캐릭터 중 하나로 자리매김했습니다.

AIMessageChunk(content='"Iron Man"은 2008년에 개봉한 마블 시네마틱 유니버스의 첫 번째 영화로, 로버트 다우니 주니어가 토니 스타크/아이언맨 역으로 출연했습니다. 이 영화는 토니 스타크가 철갑수트를 만들어 슈퍼히어로로 변신하는 이야기를 다루고 있습니다. 감독은 존 파브로가 맡았으며, 영화는 큰 성공을 거두었고 마블 시네마틱 유니버스의 시작을 알린 작품으로 손꼽힙니다. 아이언맨은 많은 팬들에게 사랑받는 마블의 대표적인 캐릭터 중 하나로 자리매김했습니다.')

In [ ]:
# 다양한 예제를 하나의 체인으로 묶음 

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
# 캐싱 : 이전에 질문 답변에 대해 캐시로 담고 있어서 이전보다 빠르게 처리

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

In [ ]:
# open Ai 비용 확인 방법

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

In [ ]:
# conversationBufferMemory : 단순하게 모든 대화 내용을 저장 (비효율적임)

from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

In [ ]:
#conversationBufferWindowMemory : 최근 대화만 저장 (개수는 사용자가 정할 수 있음)

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

In [ ]:
# conversationSummaryMemory : 요약, 처음에는 오래 걸리지만 진행될 수록 대화 내용이 많아지면서 연결된다.

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")
get_history()

In [ ]:
# conversationSummaryBufferMemory : 최근의 내용은 저장하고 오래된 것은 요약을 하여 사용한다.


from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
# conversatuinKGMemory : 질문할 때 객체를 뽑아서 질문함

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]  # load_memory_variable : 메모리 업로드


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(       # save_context : 사람과 Ai의 메시지인 input output을 메모리에 저장
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [39]:
# 영화 제목 이모티콘으로 답장 받기 (메모리 + LCEL)

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


emoji_movie_examples = [
    {
        "movie" : "탑건",
        "answer": """
            🛩️👨‍✈️🔥
        """
    },
    {
        "movie" : "대부",
        "answer": """
            👨‍👨‍👦🔫🍝
        """
    },
     {
        "movie" : "모아나",
        "answer": """
           🌊🚣‍♀️🌺
        """
    },
]

movie_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}에 대한 영화 정보를 알려줘"),
        ("ai", "{answer}")
    ]
)

movie_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=movie_example_prompt,
    examples=emoji_movie_examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 정보에 대해 잘 알고 있어, 이모지 3개로만 답변해줘"),
        movie_example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"] 


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(    
        {"input": movie},   
        {"output": result.content},
    )
    print(result)   

In [36]:
invoke_chain("모아나")


🌊🚣‍♀️🌺content='🌊🚣\u200d♀️🌺'


In [41]:
invoke_chain("인사이드아웃")

👧🧠🤔content='👧🧠🤔'


In [29]:
invoke_chain("데드풀")

🦸‍♂️💀🔫content='🦸\u200d♂️💀🔫'


In [42]:
load_memory("먼저 질문한 영화 알려줘")

[HumanMessage(content='인사이드아웃'), AIMessage(content='👧🧠🤔')]

In [2]:
# RAG : retrieval augmented generation 검색-증강 생성, 모델이 이미 학습한 데이터 이외의 데이터를 제공해서 좀 더 정교한 답변을 준다.
# Unstructured File : txt, ppt, html 등을 한번에 처리할 수 있는 loader
# recuriveCharacterTextSplitter, characterTextSplitter

# load -> transform (split) -> embed -> store (백터 데이터베이스) -> retrieve

# embed : 문자들을 숫자로 치환하여 차원(백터)을 가지고 계산한다. (연관성이 높은 단어를 찾을 수 있다. : 영화 추천 시스템)


from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")
loader = UnstructuredFileLoader("./files/document.txt")

# splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
splitter = CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)


docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

# off-the-shelf : 이미 만들어진 체인 
# chain_type : refine(정제) 여러개의 문서를 확인하여 안에서 또 질문을 해서 업데이트 시켜준다.
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

'Victory Mansions is a building where Winston Smith lives in George Orwell\'s novel "1984." It is a dilapidated apartment complex in a totalitarian society controlled by the Party. The building is described as run-down, with peeling paint, faulty plumbing, and an overall sense of neglect and decay. Despite its name, Victory Mansions do not provide a sense of victory or comfort to its residents, reflecting the oppressive and austere nature of the society in which they live.'

In [6]:
vectorstore.similarity_search("where does winston live")

[Document(page_content="Under the table Winston's feet made convulsive movements. He had not stirred from his seat, but in his mind he was running, swiftly running, he was with the crowds outside, cheering himself deaf. He looked up again at the portrait of Big Brother. The colossus that bestrode the world! The rock against which the hordes of Asia dashed themselves in vain! He thought how ten minutes ago--yes, only ten minutes--there had still been equivocation in his heart as he wondered whether the news from the front would be of victory or defeat. Ah, it was more than a Eurasian army that had perished! Much had changed in him since that first day in the Ministry of Love, but the final, indispensable, healing change had never happened, until this moment.", metadata={'source': './files/document.txt'}),
 Document(page_content="Winston thought. 'By making him suffer,' he said.", metadata={'source': './files/document.txt'}),
 Document(page_content='filled his glass up with Victory Gin, 

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
         MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    x = memory.load_memory_variables({})
    return x["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)

chain.invoke("Is Aaronson guilty")

AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.', additional_kwargs={}, response_metadata={'token_usage': <OpenAIObject at 0x1e3a9685850> JSON: {
  "prompt_tokens": 2122,
  "completion_tokens": 20,
  "total_tokens": 2142,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76b78b5e-d6bd-4521-8fd7-8b04bb44edb2-0')

In [20]:
chain.invoke("What message did he write in the table")

AIMessage(content='He wrote "2+2=5" in the dust on the table.', additional_kwargs={}, response_metadata={'token_usage': <OpenAIObject at 0x1e3a9603170> JSON: {
  "prompt_tokens": 2220,
  "completion_tokens": 16,
  "total_tokens": 2236,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e510fa9-bdce-49eb-9af7-8ed9bc7905a6-0')

In [21]:
chain.invoke("Who is Julia")

AIMessage(content='Julia is a character mentioned in the text. She is someone who Winston loves and cares for deeply, and he expresses a strong desire to protect her from harm.', additional_kwargs={}, response_metadata={'token_usage': <OpenAIObject at 0x1e3a9684b30> JSON: {
  "prompt_tokens": 2074,
  "completion_tokens": 33,
  "total_tokens": 2107,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-83fef15b-5c91-44a0-b7e5-cf2180c26534-0')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
                "documents": retriever,
                "question": RunnablePassthrough(),
            } | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer.
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind( # auto로 주면 알아서 결정해서 씀 내가 만든 걸 쓸 수도 있고 아닐 수도 있음
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:

from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)